In [1]:
import torch
import json
import numpy as np
from evaluation import *
from NN_Models import *
import random
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
y_test = torch.load('data/y_test.pt')
y_train = torch.load('data/y_train.pt')

In [3]:
# year & venue
f = open("outputs/year_venue_test.json", 'r')
year_venue_test = json.load(f)

f = open("outputs/year_venue_train.json", 'r')
year_venue_train = json.load(f)

# abstract & title
f = open("outputs/abstract_title_test.json", 'r')
abstracts_title_test = json.load(f)

f = open("outputs/abstract_title_train.json", 'r')
abstracts_title_train = json.load(f)

# coauthor
f = open("outputs/author_test.json", 'r')
author_test = json.load(f)

f = open("outputs/author_train.json", 'r')
author_train = json.load(f)

## Weighted

In [4]:
#################################
weight_sentence = 0.31982275074419797  # abstracts - title
weight_author = 0.6143938170885868  # authors
weight_year_venue = 0.09287086258899986  # year - venue

# weight_sentence = 1
# weight_author = 1
# weight_year_venue = 1

sum_ = weight_author + weight_sentence + weight_year_venue
#################################

YEAR_VENUE_WEIGHT = weight_year_venue/sum_
COAUTHOR_WEIGHT = weight_author/sum_
SENTENCE_WEIGHT = weight_sentence/sum_    

In [5]:
THRESHOLD = 0.2

y_pred_list = predict(
            author=author_test,
            year_venue=year_venue_test,
            abstracts_title=abstracts_title_test,
            YEAR_VENUE_WEIGHT=YEAR_VENUE_WEIGHT,
            COAUTHOR_WEIGHT=COAUTHOR_WEIGHT,
            SENTENCE_WEIGHT=SENTENCE_WEIGHT,
            THRESHOLD=THRESHOLD
        )

In [6]:
y_test_list = to_list(y_test)

print_scores(y_test_list, y_pred_list)

=========================Evaluation results=========================
The accuracy score of prediction is : 0.6182760855476345
The recall   score of prediction is : 0.6182760855476345
The f1       score of prediction is : 0.6248107838542082


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Model

In [4]:
clf = LogisticRegressionPredictModel()
clf.train(author_train, year_venue_train, abstracts_title_train, y_train)

100%|██████████| 6268/6268 [00:03<00:00, 1667.85it/s]


Score :  1.0


LogisticRegression(random_state=0)

In [5]:
y_pred_list = clf.evaluation(author_test, year_venue_test, abstracts_title_test)

100%|██████████| 3086/3086 [00:22<00:00, 136.73it/s]


In [6]:
y_test_list = to_list(y_test)

print_scores(y_test_list, y_pred_list)

=========================Evaluation results=========================
The accuracy score of prediction is : 0.3535320803629294
The recall   score of prediction is : 0.3535320803629294
The f1       score of prediction is : 0.29945946574050825


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
y_test_list[:5]

['8', '31', '37 41', '-1', '41']

In [7]:
y_pred_list

['-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '75',
 '-1',
 '6',
 '-1',
 '-1',
 '59',
 '-1',
 '-1',
 '-1',
 '-1',
 '92',
 '-1',
 '-1',
 '-1',
 '74',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '98',
 '-1',
 '-1',
 '27',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '85',
 '-1',
 '-1',
 '89',
 '-1',
 '-1',
 '59',
 '-1',
 '-1',
 '-1',
 '92',
 '-1',
 '-1',
 '57 68 98',
 '57',
 '-1',
 '-1',
 '83',
 '-1',
 '-1',
 '36',
 '-1',
 '25',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '22',
 '-1',
 '4',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '59',
 '-1',
 '-1',
 '-1',
 '27',
 '-1',
 '55',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '11',
 '-1',
 '-1',
 '86',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '53',
 '-1',
 '-1',
 '-1',
 '-1',
 '36',
 '25',
 '78',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '7 76',
 '-1',
 '-1',
 '0',
 '-1',
 '-1',
 '98',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '-1',
 '6',
 '-1',
 '-1',
 '-1',
 '-1',
 '21',
 '47',
 '-1',
 '-1',
 '-1',
 '-1',
 '45',
 '